In [ ]:
import os
import pandas as pd
import shutil
import re
from xml.etree import ElementTree
from collections import Counter

In [ ]:
# Функция рекурсивного поиска файлов во вложенных папках. Находит файлы и их размер, путь файла
# dir: str, ext: list
# https://stackoverflow.com/a/59803793/2441026
def run_fast_scandir(dir, ext):
    subfolders, files = [], []

    for f in os.scandir(dir):
        if f.is_dir():
            subfolders.append(f.path)
        if f.is_file():
            if os.path.splitext(f.name)[1].lower() in ext:
                files.append([f.name,f.stat().st_size,f.path])

    for dir in list(subfolders):
        sf, f = run_fast_scandir(dir, ext)
        subfolders.extend(sf)
        files.extend(f)
    return subfolders, files

# Удаление найденных файлов с ФС
# files_list: list['name','size','path']
def delete_files(files_list):
  i = 0
  for item in files_list:
    file_path = item[2]
    os.remove(file_path)
    i += 1
  return i

# Поиск пустых файлов по размеру
# files_list: list['name','size','path'], size: str
def file_size_selection(files_list, size):
  empty_files = []
  for empty_file in files_list:
    if size in empty_file[1]:
      empty_files.append(empty_file)
  return empty_files

# Переименование файлов, путем убирания последних 8 символов
# counters_list: list['name','size','path']
# https://stackoverflow.com/questions/15478127/remove-final-character-from-string
def rename_counters(counters_list):
  for counter_item in counters_list:
    filename = counter_item[2]
    os.rename(filename,filename[:-8])

# Копирование файлов в отдельный каталог
# https://stackoverflow.com/questions/15329223/copy-a-file-into-a-directory-with-its-original-leading-directories-appended
def copy_files_whith_path(src, dest_folder, dir_offset=0):
    ''' Copies src tree into dest, offset (optional) omits n folders of the src path'''
    prev_offset = 0 if dir_offset == 0 else src.replace('/', '%', dir_offset - 1).find('/') + 1
    post_offset = src.rfind('/')

    src_dirs = '' if post_offset == -1 else src[prev_offset:post_offset]
    src_filename = src[post_offset + 1:]

    os.makedirs(f'{dest_folder}/{src_dirs}', exist_ok=True)
    shutil.copy(src, f'{dest_folder}/{src_dirs}/{src_filename}')

# Поиск файлов в каталоге и вывод результатов поиска в консоль
# file_name: str, files_list: list
def find_file_by_name(file_name, files_list):
  for desc in files_list:
    if file_name in desc[0]:
      print(desc)

def simple_print_file_path(file_name):
  files = {
    'migration': get_files_lists('migration',['.desc']),
    'git': get_files_lists('git',['.desc'])
  }
  
  if isinstance(file_name, set):
    for item in list(file_name):
      find_file_by_name(item, files['migration'])
      find_file_by_name(item, files['git'])
  elif isinstance(file_name, list):
    for item in file_name:
      find_file_by_name(item, files['migration'])
      find_file_by_name(item, files['git'])
  else :
    for item in [file_name]:
      find_file_by_name(item, files['migration'])
      find_file_by_name(item, files['git'])

def simple_print_edit_path(file_name):
  files = {
    'migration': get_files_lists('migration',['.edit']),
    'git': get_files_lists('git',['.edit'])
  }
  
  if isinstance(file_name, set):
    for item in list(file_name):
      find_file_by_name(item, files['migration'])
      find_file_by_name(item, files['git'])
  elif isinstance(file_name, list):
    for item in file_name:
      find_file_by_name(item, files['migration'])
      find_file_by_name(item, files['git'])
  else :
    for item in [file_name]:
      find_file_by_name(item, files['migration'])
      find_file_by_name(item, files['git'])

def simple_print_lc_path(file_name):
  files = {
    'migration': get_files_lists('lc_migration',['.lc']),
    'git': get_files_lists('lc_git',['.lc'])
  }
  
  if isinstance(file_name, set):
    for item in list(file_name):
      find_file_by_name(item, files['migration'])
      find_file_by_name(item, files['git'])
  elif isinstance(file_name, list):
    for item in file_name:
      find_file_by_name(item, files['migration'])
      find_file_by_name(item, files['git'])
  else :
    for item in [file_name]:
      find_file_by_name(item, files['migration'])
      find_file_by_name(item, files['git'])

def get_set_from_df(df):
  result_set = set(df.values.flatten().tolist())
  return result_set

Анализ формуляров проекта

In [ ]:
# Поиск одноименных файлов в клонированном репозитории гита
# Путь поиска для клонированного гит репозитория ../../exp-03-supp/ относительно директории мигрированного проекта ../migration/exp-03-supp/
# get_files: list['name','size','path']
# file_extension: list
# target: {'git','migration'}
def get_files_lists(target, file_extension):
  folders = {
      'git': run_fast_scandir(GIT_DOCS_DIR, file_extension),
      'migration': run_fast_scandir(MIG_DOCS_DIR, file_extension),
      'lc_git': run_fast_scandir(GIT_LC_DIR, file_extension),
      'lc_migration': run_fast_scandir(MIG_LC_DIR, file_extension),
  }
  get_files = folders[target]
  return tuple(get_files[1])

# Подсчет и получение списка файлов
def print_files_amount(target, file_extension):
  files_list = get_files_lists(target, file_extension)
  print(f'Всего файлов {file_extension} в проекте {target}: {len(files_list)}')

In [ ]:
# сделать из вложенного списка [[1,2],[2,3]], плоский список[1,2,2,3]
# в список включить только названия файлов
# https://stackoverflow.com/questions/3462143/get-difference-between-two-lists
# target: project, EXP, dicts
def get_pre_set(set_name):
  files = {
   'git_flat': [sublist[0] for sublist in get_files_lists('git',['.desc'])],
   'migration_flat': [sublist[0] for sublist in get_files_lists('migration',['.desc'])],
   #'git_EXP_flat': [sublist[0] for sublist in get_files_lists('git.EXP',['.desc'])],
   #'git_dicts_flat': [sublist[0] for sublist in get_files_lists('git.dicts',['.desc'])],
   #'migration_EXP_flat': [sublist[0] for sublist in get_files_lists('migration.EXP',['.desc'])],
   #'migration_dicts_flat': [sublist[0] for sublist in get_files_lists('migration.dicts',['.desc'])]
    }

  pre_sets = {
      'git_total': files['git_flat'],
      'git_unique': set(files['git_flat']),
      'git_only': set(files['git_flat'])^(set(files['git_flat'])&set(files['migration_flat'])),
      'migration_total': files['migration_flat'],
      'migration_unique': set(files['migration_flat']),
      # https://stackoverflow.com/questions/9835762/how-do-i-find-the-duplicates-in-a-list-and-create-another-list-with-them
      # используем подсчет количества элементов в списке через Counter, если count > 1 то возвращаем элемент desc     
      'duplicated_list': [desc for desc, count in Counter(files['git_flat']).items() if count > 1],
      'duplicated': set([desc for desc, count in Counter(files['git_flat']).items() if count > 1]),
      'difference': set(files['git_flat']) ^ set(files['migration_flat']), #print(f'Найдено уникальных формуляров проекта: {len( )}')
      'the_same': set(files['git_flat']) & set(files['migration_flat']), #print(f'Найдено одноименных формуляров проекта: {len( )}')
      'new_to_git': (set(files['git_flat']) & set(files['migration_flat']))^set(files['migration_flat']), # print(f'Найдено новых для гита формуляров в мигрируемом проекте: {len( )}')
      #'difference_EXP': set(files['git_EXP_flat']) ^ set(files['migration_EXP_flat']),
      #'the_same_EXP': set(files['git_EXP_flat']) & set(files['migration_EXP_flat']),
      #'new_to_git_EXP': (set(files['git_EXP_flat']) & set(files['migration_EXP_flat']))^set(files['migration_EXP_flat']),
      #'difference_dicts': set(files['git_dicts_flat']) ^ set(files['migration_dicts_flat']),
      #'the_same_dicts': set(files['git_dicts_flat']) & set(files['migration_dicts_flat']),
      #'new_to_git_dicts': (set(files['git_dicts_flat']) & set(files['migration_dicts_flat']))^set(files['migration_dicts_flat']),
  }

  return pre_sets[set_name]

# Анализ формуляров из default.navigation

In [ ]:
# Функция проверки работы с файлом навигации. Возвращает количество folder в навигации.
def test_etree():
# Вызов экземпляра ElementTree()
  xml_tree=ElementTree.ElementTree()

# Получение файла навигации
  navigation_file = NAVIGATION_DIR
  desc_file = f'{MIG_DIR}sufd-services.EXP/setup/M_A/1.0/M_A.desc'
  lc_file = f'{MIG_DIR}T_c/T_c.lc'

  xml_tree.parse(lc_file)
  root = xml_tree.getroot()

#проверка полученного xml
#et.dump(root)

#проверка показала, что у каждого элемента есть префикс неймспейса ns0
#задаем dict для неймпспейсов
  ns = {'ns0': '{http://ufos.otr.ru/schema/navigation/v2}'}
#пример получения пути для неймспейса ns0
#print(ns['ns0'])

#Пример поиска тега folder по xml с неймспейсом
  #folders = root.findall(".//{http://www.otr.ru/sufd/document/desc}folders")
  #len(folders)
  return ElementTree.dump(root)

# Функция возвращает root для xml файла
def get_xml_root(file_type, elem, xml_ns, path=None):
  files = {
      'navigation': NAVIGATION_DIR,
      'path_generator': path
  }
  ns = {
      'navigation': '{http://ufos.otr.ru/schema/navigation/v2}',
      'desc': '{http://www.otr.ru/sufd/document/desc}',
      'lc': '{http://www.otr.ru/sufd/document/life-cycle}',
      'studio': '{http://www.otr.ru/sufd/document/studio}'
      }
  elements = {
        'nav_folder': 'folder',
        'nav_formular': 'document-type',
        'desc_field': 'field',
        'lc_transition': 'transition',
        'lc_state': 'state'
      }
  xml_tree=ElementTree.ElementTree()
  xml_tree.parse(files[file_type])
  root = xml_tree.getroot()
  if elem == 'desc_lc':
    xml_elements = root.get('lifeCycle')
  elif elem == 'lc_parent':
    xml_elements = root.get('parent-id')
  else: 
    xml_elements = tuple(
        root.findall(f'.//{ns[xml_ns]}{elements[elem]}')
    )
  #print(f'Всего элементов {elements[elem]}: {len(xml_elements)}')
  #print(f'Пример элемента: {xml_elements[0]}')
  return xml_elements

In [ ]:
def navigation_df(xml_elements):
  formulars = []
  
  for i,val in enumerate(xml_elements):
    id = val.get('type')
    formular = {id}
    formulars.append(formular)
  
  df = pd.DataFrame(formulars, columns=['formular'], dtype='category')
  df = df.drop_duplicates('formular',keep='last') #Получение списка уникальных формуляров (исключение дубликатов, которые образовались из-за обилия фильтр-папок)

  return df

In [ ]:
def calc_navigation_df(target):
  df = navigation_df(get_xml_root('navigation','nav_formular','navigation'))
  df['desc'] = df.formular.transform(lambda x: x+'.desc').astype('category') # представим, что все формуляры в навигации имеют расширение .desc
  df['ext'] = df.formular.transform(lambda x: x+'.ext.desc').astype('category') # представим, что все формуляры в навигации имеют расширение .ext.desc

  df_migration_desc = pd.DataFrame (
      get_pre_set('migration_unique'),
      columns = ['formular'],
      dtype='category')
  
  df_git_desc = pd.DataFrame (
      get_pre_set('git_unique'),
      columns = ['formular'],
      dtype='category')
  
  base_formulars = {'D_E.desc', 'D_E.ext.desc', 'D_SE.desc', 'D_SE.ext.desc', 'R_SE.desc', 'R_SE.ext.desc'}
  df_base = pd.DataFrame(
      base_formulars,
      columns = ['formular'],
      dtype='category')
  
  df_base_mig = df_migration_desc[df_migration_desc.formular.isin(df_base.formular)]
  df_base_git = df_git_desc[df_git_desc.formular.isin(df_base.formular)]

  df_base_to_copy = df_base_mig[df_base_mig.formular.isin(df_base_git.formular)] # базовые формуляры, которые можно мигрировать по стандартному сценарию
  df_base_to_find_in_binary = df_base_git[~df_base_git.formular.isin(df_base_mig.formular)] # базовые формуляры, которые надо искать в бинарках

  # Формуляры, которые есть и в навигации, и в мигрируемом проекте
  # https://towardsdatascience.com/the-unreasonable-effectiveness-of-method-chaining-in-pandas-15c2109e3c69
  df_AND_navigation_migration = (
    df[df.desc.isin(df_migration_desc.formular)]
    .loc[:, ['desc']] # список формуляров с расширением .desc
    .rename(columns={'desc': 'formular'}) # переименовываем колонку в formular
  )

  df_AND_navigation_migration_ext = (
      df[df.ext.isin(df_migration_desc.formular)]
      .loc[:, ['ext']]
      .rename(columns={'ext': 'formular'})
  )

  # Объединение desc и ext.desc
  df_full_navigation_migration = pd.concat([df_AND_navigation_migration,df_AND_navigation_migration_ext, df_base_to_copy])

  # Аналогично для гита
  df_join_navigation_git = (
      df[df.desc.isin(df_git_desc.formular)]
      .loc[:, ['desc']] 
      .rename(columns={'desc': 'formular'})
  )

  df_AND_navigation_git_ext = (
      df[df.ext.isin(df_git_desc.formular)]
      .loc[:, ['ext']]
      .rename(columns={'ext': 'formular'})
  )
  
  df_full_navigation_git = pd.concat([df_join_navigation_git,df_AND_navigation_git_ext, df_base_to_copy])

  # Формуляры из навигации, которые есть в мигрируемом проекте, но которых вообще нет в гите
  # Надо копировать по полному сценарию
  df_1 = df_full_navigation_migration[~df_full_navigation_migration.index.isin(df_full_navigation_git.index)]
  
  # Формуляры из навигации, которые есть в мигрируемом проекте и в гите
  # Надо копировать по стандартному сценарию
  df_2 = df_full_navigation_migration[df_full_navigation_migration.index.isin(df_full_navigation_git.index)]

  # Поиск формуляров для ручного копирования в виду несоотвествия модели desc/ext.desc
  df_m1 = df_migration_desc[df_migration_desc.formular.isin(df_2.formular)]
  df_m2 = df_git_desc[df_git_desc.formular.isin(df_2.formular)]
  df_5 = df_m1[~df_m1.formular.isin(df_m2.formular)]

  # Надо копировать по стандартному сценарию, убраны формуляры для ручного копирования
  df_6 = df_2[~df_2.formular.isin(df_5.formular)]

  # Формуляры из навигации, которых нет в мигрируемом проекте, но есть в гите:
  df_3 = df_full_navigation_git[~df_full_navigation_git.index.isin(df_full_navigation_migration.index)]
  df_4 = pd.concat([df_3,df_base_to_find_in_binary])

  result ={
      'nav_total_migration': df_full_navigation_migration,
      'nav_total_git': df_full_navigation_git,
      'nav_to_full_copy': df_1,
      'nav_to_copy': df_6,
      'to_manual_copy': df_5,
      'nav_to_find_in_binary': df_4,
      'base_to_copy': df_base_to_copy,
      'base_to_find_in_binary': df_base_to_find_in_binary
  }

  return result[target]

## Подготовка к миграции. Копирование формуляров

In [ ]:
def run_fast_scandir2(dir, ext):
    subfolders, files = [], []

    for f in os.scandir(dir):
        if f.is_dir():
          subfolders.append(f.path)
        if f.is_file(): 
          if  os.path.splitext(f.name)[1].lower() in ext:
            files.append(f.name,f.stat().st_size,f.path)

    for dir in list(subfolders):
        sf, f = run_fast_scandir(dir, ext)
        subfolders.extend(sf)
        files.extend(f)
    return tuple(files)

In [ ]:
# генератор полей из desc
def desc_fields_generator(xml_elements): 
  for i,val in enumerate(xml_elements):
    id = val.get('id')
    yield id

In [ ]:
def gen_xml_elements(target):
  folders = {
      'git': GIT_DOCS_DIR,
      'migration': MIG_DOCS_DIR
  }

  for item_list in run_fast_scandir2(folders[target], ['.desc']):
    if item_list[0] != '.DS_Store': #спец проверка для macOS на предмет системного файла Finder
      l = desc_fields_generator(
        get_xml_root('path_generator','desc_field','desc', item_list[-1])
      )
      yield item_list[0], len(tuple(l))

In [ ]:
# трансформатор списка полей в df
def desc_df(target):
  df = pd.DataFrame.from_records(
     tuple(gen_xml_elements(target)),
     columns=['formular','fields_num']
  )
  return df

In [ ]:
# Новая версия функции, которая учитывает вычисленный с формулярами calculate_df
def prepare_df2():
  df_mig = pd.DataFrame(
      get_files_lists('migration',['.desc']), 
      columns = ['formular', 'size', 'path'])
  
  df_git = pd.DataFrame(
      get_files_lists('git',['.desc']),
      columns = ['formular', 'size', 'path'])
  
  df_m1 = df_mig[df_mig.formular.isin(calc_navigation_df('nav_to_copy').formular)]
  df_m2 = df_git[df_git.formular.isin(calc_navigation_df('nav_to_copy').formular)]
  df_m3 = pd.merge(df_m1, df_m2, on='formular', how='outer')
  df_m4 = df_m3[~df_m3.formular.isin(calc_navigation_df('nav_to_full_copy').formular)] #исключение из списка формуляров для ручного копирования


  #поиск наибольшего файла среди дублей
  df_1 =  pd.DataFrame(
    get_pre_set('duplicated'),
    columns=['formular']) #получение df с дубликатами
  df_2 = pd.merge(df_m4, df_1, on='formular')
  df_2['size_rate'] = df_2['size_y']/df_2['size_x']
  df_2['size_max'] = df_2.groupby('formular')['size_rate'].transform('max')
  df_3 = df_2[lambda x: x.size_rate != x.size_max] #df с неподходящими дублирующимися формулярами

  df = df_m4[~df_m4.path_y.isin(df_3.path_y)] #исключение неподходящих дублирующих формуляров из общего списка
  
  return df

In [ ]:
def copy_file_to_git(df):
  i = 0
  for row in df.iterrows():
    folders_list_x = row[1].path_x.split('/')[:-1]
    new_path_x = os.path.join('/',*folders_list_x)[1:]+'/' #* унарный оператор нужен, чтобы распаковать список https://www.python.org/dev/peps/pep-0448/, slice [1:] нужен чтобы удалить первую / из path
    folders_list_y = row[1].path_y.split('/')[:-1]
    new_path_y = os.path.join('/',*folders_list_y)[1:]+'/'
    for item in os.listdir(new_path_x):
      if not re.search('edit', item): #исключение edit из списка копирования
        i += 1
        shutil.copy2(new_path_x+item, new_path_y+item)
        print(i, item)

## Подбор и копирование ВФ

In [ ]:
def change_edit(path_edit_git, path_edit_mig):
  xml_tree=ElementTree.ElementTree()
  ElementTree.register_namespace('','http://www.otr.ru/sufd/document/viewform')
  ElementTree.register_namespace('st','http://www.otr.ru/sufd/document/studio')
  
  xml_tree.parse(path_edit_git)
  root = get_root(path_edit_git)

  ns = '{http://www.otr.ru/sufd/document/viewform}'

  for tags in xml_elem_generator('path_generator', 'edit_tags','studio', path_edit_mig):
    if tags:
      st_tags = ElementTree.Element('st:tags')
      st_tags.extend(tags)
      ElementTree.indent(st_tags, space=' ', level=0)
      root.insert(0,st_tags)
  
  actions = actionsContentHandler('actions', path_edit_mig) # блок <actions> из мигрируемого документа
  actions_git = root.findall(ns+'actions') # блок <actions> из гит документа
  if actions:
    for group in actions.findall(ns+'actions-group'):
      actions_id_mig = [_.get('id') for _ in group.iter(ns+'action')]
      composite_id_mig = [_.get('id') for _ in group.iter(ns+'composite-action')]
      # проверка на соотвествие действия гита и миграции
      #for action_mig in group.iter(ns+'action'):
        #for action_git in actions_git[0].iter(ns+'action'):
        #  if action_mig.get('id') == action_git.get('id'):
        #    action_mig = action_git
      actions_git[0].insert(0,group)
      delete_actions(actions_id_mig, actions_git[0])
      delete_actions(composite_id_mig, actions_git[0])
    actions.clear()
    
  # меняется полностью блок условий на ВФ
  for conditions in xml_elem_generator('path_generator', 'edit_conditions', 'edit', path_edit_mig):
    c = {k:condition for k,condition in enumerate(root) if condition.tag == ns+'conditions'}
    index_tag = next(iter(c))
    root.remove(c[index_tag])
    c_ = ElementTree.Element(ns+'conditions')
    c_.extend(conditions)
    root.insert(index_tag,c_)
    
  
  # нужно, чтобы заменить кавычки в тексте, но не в атрибутах тегов
  for elem in root.iter():
    if elem.text and not re.match('\s', elem.text):
      elem.text = elem.text.replace('"', '&quot;')
  
  xml_tree.write(path_edit_git, encoding='utf-8', xml_declaration = True)
  root.clear()


In [ ]:
def get_root(path):
    xml_tree = ElementTree.ElementTree()
    ElementTree.register_namespace("", "http://www.otr.ru/sufd/document/viewform")
    ElementTree.register_namespace("st", "http://www.otr.ru/sufd/document/studio")

    try:
        xml_tree.parse(path)
    except Exception as e:
        print(e, path)
    # except ElementTree.ParseError as err:
    #    print(err, path)
    else:
        return xml_tree.getroot()

In [ ]:
# https://stackoverflow.com/questions/6847263/search-and-remove-element-with-elementtree-in-python
# Не можем удалить последовательно два элемента в xml
# Since each element is an index of Element._children you shouldn't delete while forward iterating
def delete_actions(actions, parents):
    for child in reversed(parents):
      if child.get('id') in actions:
        parents.remove(child)

In [ ]:
def del_whitespace_edit(path_edit_git):
  new_file = ''
  with open(path_edit_git, 'r', encoding='utf-8') as f:
    for line in f:
      line = re.sub('\" />', '\"/>', line)
      line = re.sub(' />', '/>', line)
      line = re.sub('&amp;quot;', '&quot;', line)
      line = re.sub('&gt;=', '>=', line)
      new_file = new_file + utf8_replace_edit(line)
    f.close()

  fout = open(path_edit_git, 'w')
  fout.write(new_file)
  fout.close()

def utf8_replace_edit(line):
    if re.match('<\?xml version', line):
      replaced_line = re.sub('\'', '\"', line)
      replaced_line = re.sub('utf', 'UTF', replaced_line)
      return replaced_line
    return line

In [ ]:
def actionsContentHandler(tag, file_path):
  #file_path = './migration/MSC/Func/docs/EXP/setup/E_O/1.0/E_O.edit'
  context = ElementTree.iterparse(file_path, events=("start", "end"))
  ns = '{http://www.otr.ru/sufd/document/viewform}'
  _, root = next(context)

  for event, elem in context:
    if event == 'end' and elem.tag == ns+tag:
      for child in elem:
        if child.tag == ns+'action' or child.tag == ns+'action-group':
          return elem
  root.clear()

In [2]:
def prepare_edits_list(df):
  for row in df.iterrows():
    folders_list_x = row[1].path_x.split('/')[:-1]
    new_path_x = os.path.join('/',*folders_list_x)[1:]+'/' #* унарный оператор нужен, чтобы распаковать список https://www.python.org/dev/peps/pep-0448/, slice [1:] нужен чтобы удалить первую / из path
    folders_list_y = row[1].path_y.split('/')[:-1]
    new_path_y = os.path.join('/',*folders_list_y)[1:]+'/'
    for item in os.listdir(new_path_x):
        if re.search('edit', item) and os.path.isfile(new_path_y+item):
            print(new_path_x+item)
            tags = {tag for tag in xml_elem_generator('path_generator', 'edit_tags', 'studio', new_path_x+item)}
            actions = {action for action in xml_elem_generator('path_generator', 'edit_action_group','edit', new_path_x+item)}
            conditions = {condition for condition in xml_elem_generator('path_generator', 'edit_condition_group','edit', new_path_x+item)}
            print(tags, actions, conditions)
            if tags^actions^conditions:
                yield (new_path_x+item, new_path_y+item)

In [ ]:
# <Element '{http://www.otr.ru/sufd/document/studio}tags' at 0x1207f14f0>,
#  './migration/MSC/Func/docs/DICTS/setup/E_y/1.0/E_y.edit',
#  'E_y.edit
def batch_edit(df):
 i = 0
 for e in prepare_edits_list(df):
    i += 1
    path_edit_git = e[-1]
    path_edit_mig = e[0]
    item = path_edit_git.split("/")[:]
    print(i, item[-1], path_edit_git)
    change_edit(path_edit_git, path_edit_mig)
    del_whitespace_edit(path_edit_git)

## Парсинг desc и подсчет кол-ва полей

In [ ]:
# трансформатор списка полей в df
def desc_df(target):
  df = pd.DataFrame.from_records(
     tuple(gen_xml_elements(target)),
     columns=['formular','fields_num']
  )
  return df

# Сравнение количества полей в формулярах
def compare_fields_count(target):
  df_nav = calc_navigation_df('nav_to_copy')
  df = (
      pd.merge(desc_df('git'), desc_df('migration'), on='formular')
      .drop_duplicates('formular',keep='last')
  )
  df = df[df.formular.isin(df_nav.formular)]
  result ={
      'have_diff': df[lambda x: x.fields_num_x != x.fields_num_y], #находим формуляры с разным числом полей
      'the_same_fields_count': df[lambda x: x.fields_num_x == x.fields_num_y],
      'git_have_big': df[lambda x: x.fields_num_x > x.fields_num_y],
      'migration_have_big': df[lambda x: x.fields_num_x < x.fields_num_y],
  }
  return result[target]

In [ ]:
def desc_fields(target):
  folders = {
      'git': GIT_DIR,
      'migration': MIG_DIR
  }

  for item_list in run_fast_scandir2(folders[target], ['.desc']):
    if item_list[0] != '.DS_Store': #спец проверка для macOS на предмет системного файла Finder
      l = desc_fields_generator(
        get_xml_root('path_generator','desc_field','desc', item_list[-1])
      )
      yield item_list[0], tuple(l)


def compare_fields():
  migration = tuple(desc_fields('migration'))

  for x in desc_fields('git'):
    for y in migration:
      if x[0] == y[0]:
        result = set(y[-1])^(set(x[-1])&set(y[-1]))
        if result != set():
          yield y[0], result

In [ ]:
def get_fields_from(gen, formular):
  field_list = [x[-1] for x in tuple(gen) if x[0] == formular]
  return field_list

def df_compare_fields(formular):
  df_git = pd.DataFrame(
    get_fields_from(
      desc_fields('git'), 
      formular
    )[0],
    columns = ['field_git']
  )

  df_migration = pd.DataFrame(
    get_fields_from(
      desc_fields('migration'), 
      formular
    )[0],
    columns = ['field_migration']
  )

  df_1 = df_git[~df_git.field_git.isin(df_migration.field_migration)] # покажет поля, которые есть в гите, но нет в миграции
  df_2 = df_migration[~df_migration.field_migration.isin(df_git.field_git)] # покажет поля, которые есть в миграции, но нет в гите
  df = pd.concat([df_1,df_2], axis=1,ignore_index=True)

  return df

# Поиск и копирование ЖЦ

In [ ]:
# Получение названия ЖЦ из desc файла
def generator_xml_lc(target):
  for item_list in get_files_lists(target,['.desc']):
    if item_list[0] != '.DS_Store': #спец проверка для macOS на предмет системного файла Finder
      l = get_xml_root('path_generator','desc_lc','desc', item_list[-1])
      yield item_list[0], l

# Получение названия родительского ЖЦ из lc файла
def generator_parent_lc(target):
  for item_list in get_files_lists(target,['.lc']):
    if item_list[0] != '.DS_Store': #спец проверка для macOS на предмет системного файла Finder
      l = get_xml_root('path_generator','lc_parent','lc', item_list[-1])
      yield item_list[0], l

In [ ]:
# Ищет по списку ЖЦ из формуляров гита, +все родительские ЖЦ из ЖЦ гита 
# input target: {'nav_total_migration', 'nav_total_git': df_full_navigation_git, 'nav_to_full_copy','nav_to_copy','to_manual_copy',
#'nav_to_find_in_binary', 'base_to_copy','base_to_find_in_binary'}
def prepare_lc_df(input_target, output_target):
  df_target = calc_navigation_df(input_target)
  
  df_lc = (
      pd.DataFrame(
                  generator_xml_lc('git'),
                  columns = ['formular', 'lc_name'])
      .drop_duplicates(['formular', 'lc_name'],keep='last')
      .astype({'formular': 'category', 'lc_name': 'category'}) # конвртация типа столбца для увеличения производительности
  )

  df_parent = (
      pd.DataFrame(
          generator_parent_lc('lc_git'),
          columns = ['lc','parent']
      )
      .astype({'lc': 'category', 'parent': 'category'}) # конвртация типа столбца для увеличения производительности
      .drop_duplicates('parent',keep='last')
      .loc[: , ['parent']]
      .rename(columns={'parent': 'lc'})
      .dropna()
  )

  df_1 = df_lc[df_lc.lc_name.isnull()]
  df_1 = df_1[df_1.formular.isin(df_target.formular)]
  

  df_lc['lc'] = df_lc.lc_name.transform(lambda x: x+'.lc')
  df_parent['lc'] = df_parent.lc.transform(lambda x: x+'.lc') 

  df_lc = (
      df_lc[df_lc.formular.isin(df_target.formular)]
      .astype({'lc': 'category'}) # конвртация типа столбца для увеличения производительности
      .loc[:, ['lc']]
      .drop_duplicates('lc',keep='last')
      .merge(df_parent, how='outer')
  )
  
  df_mig = (pd.DataFrame(
      get_files_lists('lc_migration',['.lc']), 
      columns = ['lc', 'size', 'path'])
    .astype({'lc': 'category', 'size': 'int32', 'path': 'category' }) # конвртация типа столбца для увеличения производительности
  )
  
  df_mig = df_mig[df_mig.lc.isin(df_lc.lc)]
  
  df_git = (pd.DataFrame(
      get_files_lists('lc_git',['.lc']),
      columns = ['lc', 'size', 'path'])
    .astype({'lc': 'category', 'size': 'int32', 'path': 'category' }) # конвртация типа столбца для увеличения производительности
  )
  
  df_git = df_git[df_git.lc.isin(df_lc.lc)]

  df = pd.merge(df_mig,df_git, on='lc')

  result = {
      'lc_to_copy': df, #список для копирования
      'lc_to_find_in_binary': df_lc[~df_lc.lc.isin(df_mig.lc)], #список ЖЦ которых нет в мигрируемом проекте
      'lc_null': df_1, #проверка на то, что у всех входящих формуляров ЖЦ задан (не пуст)
      'parent_lc': df_parent #список всех родительских ЖЦ из ЖЦ гита
  }

  return result[output_target]

In [ ]:
# генератор переходов(по id элемента) и статусов(по system-name элемента)из lc
def elems_attrib_generator(xml_elements): 
  for i,val in enumerate(xml_elements):
    if val.get('id') is not None:
      data = val.get('id')
    elif val.get('system-name') is not None:
      data = val.get('system-name')
    yield data

## Получение студийной информации из статусов и переходов

In [5]:
# https://stackoverflow.com/questions/324214/what-is-the-fastest-way-to-parse-large-xml-docs-in-python
# Изменение парсинга xml с full на lazy (функция iterparse)
def xml_elem_generator(file_path, tag, ns, path=None):
  files = {
      'navigation': NAVIGATION_DIR,
      'path_generator': path
  }
  xml_ns = {
      'navigation': '{http://ufos.otr.ru/schema/navigation/v2}',
      'desc': '{http://www.otr.ru/sufd/document/desc}',
      'lc': '{http://www.otr.ru/sufd/document/life-cycle}',
      'studio': '{http://www.otr.ru/sufd/document/studio}',
      'edit': '{http://www.otr.ru/sufd/document/viewform}'
      }
  xml_tags = {
        'nav_folder': 'folder',
        'nav_formular': 'document-type',
        'desc_field': 'field',
        'lc_transition': 'transition',
        'lc_state': 'state',
        'lc_bounds': 'bounds',
        'lc_instate':'in-state',
        'edit_tags': 'tags',
        'edit_tag': 'tag',
        'edit_action_group': 'actions-group',
        'edit_condition_group': 'conditions-container',
        'edit_conditions': 'conditions'
      }

  context = ElementTree.iterparse(files[file_path], events=("start", "end"))
  event, root = next(context)

  for event, elem in context:
    if event == 'end' and elem.tag == xml_ns[ns]+xml_tags[tag]:
      yield elem
      elem.clear()
    root.clear()

In [ ]:
# генератор переходов(по id элемента) и статусов(по system-name элемента)из lc
def elems_attrib_generator2(xml_elements): 
  if xml_elements is not None:
    for val in xml_elements:
      if val.get('id') is not None: # если у элемента есть id, то возвращаем его - это системное имя перехода
        data = val.get('id')
      elif val.get('system-name') is not None: # если у элемента есть system-name, то возвращаем его - это системное имя статуса
        data = val.get('system-name')
      yield data

In [ ]:
# генератор bounds для переходов и статусов
# xml_elem: lc_state, lc_transition
def gen_bounds(xml_elem, path_lc_git, path_lc_mig):
  st_ns = '{http://www.otr.ru/sufd/document/studio}'

  if xml_elem == 'lc_transition':
    attr = 'id'
  elif xml_elem == 'lc_state':
    attr = 'system-name'

  tr_mig = xml_elem_generator('path_generator', xml_elem,'lc', path_lc_mig)
  tr_git = elems_attrib_generator2(
    xml_elem_generator('path_generator', xml_elem,'lc', path_lc_git)
  )

  for x in tr_mig: # xml elem переходы из миграции
    if x.get(attr) in tr_git:
      bound = x.find(f'.//{st_ns}bounds')
      yield x.get(attr), bound

In [ ]:
# генератор bounds для переходов и статусов
# xml_elem: lc_state, lc_transition
def gen_instate(xml_elem, path_lc_git, path_lc_mig):
  ns = '{http://www.otr.ru/sufd/document/life-cycle}'

  if xml_elem == 'lc_transition':
    attr = 'id'
  elif xml_elem == 'lc_state':
    attr = 'system-name'

  tr_mig = xml_elem_generator('path_generator', xml_elem,'lc', path_lc_mig)
  tr_git = elems_attrib_generator2(
    xml_elem_generator('path_generator', xml_elem,'lc', path_lc_git)
  )


  for x in tr_mig: # xml elem переходы из миграции
    yield x.get(attr)
    
    if x.get(attr) in tr_git:
      for instate in x[1]:     
        if instate.get('pictogramColor'):
          yield instate.text, instate.get('pictogramColor'), 'instate'
      for outstate in x[2]:     
        if outstate.get('pictogramColor'):
          yield outstate.text, outstate.get('pictogramColor'), 'outstate'

In [ ]:
# Изменение файла ЖЦ в папке гита
def add_bounds(path_lc_git, path_lc_mig):

  xml_tree=ElementTree.ElementTree()
  ElementTree.register_namespace('','http://www.otr.ru/sufd/document/life-cycle')
  ElementTree.register_namespace('st','http://www.otr.ru/sufd/document/studio')
  xml_tree.parse(path_lc_git)
  root = xml_tree.getroot()

  ns = '{http://www.otr.ru/sufd/document/life-cycle}'

  for tr_bound in gen_bounds('lc_transition', path_lc_git, path_lc_mig):
    transition = root.find(f'.//{ns}transition[@id="{tr_bound[0]}"]')
    transition.append(tr_bound[-1])

  for tr_instate in gen_instate('lc_transition', path_lc_git, path_lc_mig):
    if isinstance(tr_instate, str):
      tr = root.find(f'.//{ns}transition[@id="{tr_instate}"]')
    elif tr_instate[-1] == 'instate': 
      for child in tr[1]:
        if child.text == tr_instate[0]:
          child.set('pictogramColor', tr_instate[1])
    elif tr_instate[-1] == 'outstate': 
      for child in tr[2]:
        if child.text == tr_instate[0]:
          child.set('pictogramColor', tr_instate[1])

  for st_bound in gen_bounds('lc_state', path_lc_git, path_lc_mig):
    state = root.find(f'.//{ns}state[@system-name="{st_bound[0]}"]')
    state.append(st_bound[-1])
 
  xml_tree.write(path_lc_git, encoding='utf-8', xml_declaration = True)

In [ ]:
# Удаление лишних пробелов " /> из xml файла
def del_whitespace(path_lc_git):
  new_file = ''
  with open(path_lc_git, 'r', encoding='utf-8') as f:
    for line in f:
      new_line = re.sub('\" />', '\"/>', line)
      new_line2 = re.sub(' />', '/>', new_line)
      new_file = new_file + utf8_replace(new_line2)
    f.close()

  fout = open(path_lc_git, 'w')
  fout.write(new_file)
  fout.close()

In [ ]:
def utf8_replace(line):
    if re.match('\s+<description>', line):
      replaced_line = re.sub('\"', '&quot;', line)
      replaced_line = re.sub('&gt;', '>', replaced_line)
      return replaced_line
    # Так как много типов параметров, то проще все смотреть, чем перечислять
    # (variable|filterQuery|FILTER_QUERY|sql|headerFilterQuery|expression|struct|sourceCriteria|group|filterTable|additionalFilter|firstTag)
    elif re.match('\s+<parameter name=\"', line):
      replaced_line = re.sub('&#13;&#10;', '&#xD;&#xA;', line)
      replaced_line = re.sub('&#09;', '&#x9;', replaced_line)
      replaced_line = re.sub('&gt;', '>', replaced_line)
      return replaced_line
    elif re.match('<\?xml version', line):
      replaced_line = re.sub('\'', '\"', line)
      replaced_line = re.sub('utf', 'UTF', replaced_line)
      return replaced_line
    return line

## Обработка массива ЖЦ

In [ ]:
def batch_lc(df):
  for idx, row in df.iterrows():
    add_bounds(row.path_y, row.path_x)
    del_whitespace(row.path_y)
    print(f'{idx}    {row.lc}')